In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, ops
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

### Useful Funcs

In [5]:
def ax_standard(ax):

    ax.grid(True, alpha=0.5)
    ax.set_xlabel("Epoch")

def plot_results(history):

    line_color = "#f07167"

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.tight_layout(pad=5)

    ax[0].plot(history["loss"], color = line_color)
    ax_standard(ax[0])
    ax[0].set_ylabel("Total Loss")


    ax[1].plot(history["kl_loss"], color = line_color)
    ax_standard(ax[1])
    ax[1].set_ylabel("KL Loss")


    ax[2].plot(history["reconstruction_loss"], color = line_color)
    ax_standard(ax[2])
    ax[2].set_ylabel("Reconstruction Loss")

In [6]:
def plot_training_vs_validation_loss(history, plot_kl_as_log = False):
    """
    Plots the total loss, reconstruction loss, and KL divergence loss
    for both training and validation.

    Parameters:
    history: Keras history object containing training and validation loss values per epoch.
    """

    fig = plt.figure(figsize = (15, 15))
    


    plt.figure(figsize=(10, 5))

    # Plot total loss
   # plt.plot(history.history["loss"], label="Train Loss", color="#545f66")
   # plt.plot(history.history["val_loss"], label="Validation Loss", color="#829399", linestyle="dashed")

    # Plot reconstruction loss
    plt.plot(history.history["reconstruction_loss"], label="Train Reconstruction Loss", color="#8BE4CB")
    #plt.plot(history.history["val_reconstruction_loss"], label="Validation Reconstruction Loss", color="#DAFA9E", linestyle="dashed")
    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.title("Training vs. Validation Reconstruction Loss")
    plt.legend()
    plt.grid(True)
    plt.show()
    # Plot KL loss
    if plot_kl_as_log:
        plt.plot(np.log(history.history["kl_loss"]), label="Train KL Loss", color="#b1cc74")
    else:
        plt.plot(history.history["kl_loss"], label="Train KL Loss", color="#b1cc74")
    #plt.plot(history.history["val_kl_loss"], label="Validation KL Loss", color="#DAFA9E", linestyle="dashed")

    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.title("Training vs. Validation Kl_loss")
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage (after training the VAE model)
# plot_training_vs_validation_loss(vae.history)  # Uncomment this after training is completed


### Generating Data

In [7]:
# Generate synthetic sine wave sequences
def generate_sine_data(num_samples=1000, sequence_length=20, noise_level=0.1):
    """
    Generate sine wave sequences with slight variations.
    num_samples: Number of sequences.
    sequence_length: Number of time steps per sequence.
    noise_level: Amplitude of added random noise.
    Returns: (num_samples, sequence_length, 1) shape dataset
    """
    X = []
    for _ in range(num_samples):
        freq = np.random.uniform(0.1, 0.5)  # Random frequency
        phase = np.random.uniform(0, np.pi)  # Random phase shift
        sequence = np.sin(np.linspace(0, 2 * np.pi * freq, sequence_length) + phase)
        sequence += np.random.normal(0, noise_level, sequence_length)  # Add noise
        X.append(sequence)

    X = np.array(X).reshape(num_samples, sequence_length, 1)  # Reshape for LSTM input
    return X

# Generate data
X_train = generate_sine_data(num_samples=1000)
X_val = generate_sine_data(num_samples=200)

# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices(X_val).batch(32)

print("Training Data Shape:", X_train.shape)  # (1000, 20, 1)
print("Validation Data Shape:", X_val.shape)  # (200, 20, 1)


Training Data Shape: (1000, 20, 1)
Validation Data Shape: (200, 20, 1)


### Building Model

In [8]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, latent_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #self.seed_generator = keras.random.SeedGenerator(3718)

        self.latent_dim = latent_dim

        self.dense_mean = layers.Dense(self.latent_dim, name="z_mean")
        self.dense_log_var = layers.Dense(self.latent_dim, name="z_log_var")


    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        time_dim = ops.shape(z_mean)[1]
        dim = ops.shape(z_mean)[2]
        epsilon = tf.keras.backend.random_normal(shape=(batch,time_dim, dim)) #batch, time steps, latent dim
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    

class Encoder(layers.Layer):

    def __init__(self, lstm_1_dim, lstm_2_dim, latent_dim,
                 *args, **kwargs):
        super.__init__(*args, **kwargs)

        self.lstm_1_dim = lstm_1_dim
        self.lstm_2_dim = lstm_2_dim
        self.latent_dim = latent_dim

        self.lstm_1 = layers.LSTM(self.lstm_1_dim, return_sequences=True)
        self.lstm_2 = layers.LSTM(self.lstm_2_dim, return_sequences=True)

        self.dense_mean = layers.Dense(self.latent_dim, name="z_mean")
        self.dense_log_var = layers.Dense(self.latent_dim, name="z_log_var")

        self.sampler = Sampling()

    def call(self, inputs):

        belief_state = self.lstm_1(inputs)
        belief_state = self.lstm_2(belief_state)

        

### Rough

In [9]:
def A_poly(A, power):
    r""""
    Returns a decresing polynomial in powers of A.
    Ex: For power = 3, returns A^3 + A^2 + A + 1
    """
    if power == 0: return np.ones_like(A)
    return A**power + A_poly(A, power-1)

In [10]:
A = np.array([-2, 2, 3])

A_poly(A, 3)

array([-5, 15, 40])